# Creat data for time series analysis 1

In [1]:
using Dates
using DataFrames: DataFrame
using Random: MersenneTwister
using JLD2: @save

include("../src/Simulation.jl")
using Main.Simulation: Param, Model, Strategy, POPULATION, PAYOFF, STRATEGY_MUTATION, RELATIONSHIP_MUTATION, interaction!, death!, birth!

include("../src/Network.jl")
using Main.Network: normalize_degree!, normalize_weight!, nv

In [2]:
function run(model::Model)::Tuple{Vector{Vector{Strategy}}, Vector{Matrix{Float16}}}
    @show model.param.variability_mode

    strategy_vec_vec = []
    weights_vec = []

    for generation = 1:(model.param.generations)
        model.generation = generation
        model.payoff_vec .= 0.0

        interaction!(model)
        death!(model, model.param.rng)
        birth!(model, model.param.rng)
        normalize_degree!(model.weights, model.param.initial_k)

        # normalize_weight!
        current_N = nv(model.weights)
        std_weight_sum = Float64(model.param.initial_w) * min(model.param.initial_k, current_N) * current_N
        normalize_weight!(model.weights, std_weight_sum)

        push!(strategy_vec_vec, copy(model.strategy_vec))
        push!(weights_vec, copy(model.weights))
    end
    
    return strategy_vec_vec, weights_vec
end;

In [3]:
trial = 10
timesteps = 6000

# Common parameters
common_params = Dict{Any, Any}(
    :initial_N => 1000,
    :initial_k => 100,
    :initial_T => 1.5,
    :S => -0.5,
    :initial_w => 0.5,
    :Δw => 0.05,
    :interaction_freqency => 1.0,
    :reproduction_rate => 0.05,
    :δ => 1.0,
    :initial_μ_s => 0.01,
    :initial_μ_r => 0.01,
    :generations => timesteps,
)

# Stable parameters
stable_params = copy(common_params)
stable_params[:β] = 0.0
stable_params[:sigma] = 0.0

# POPULATION variable parameters
population_params = copy(common_params)
population_params[:variability_mode] = POPULATION
population_params[:β] = 0.8
population_params[:sigma] = 180.0

# PAYOFF variable parameters
payoff_params = copy(common_params)
payoff_params[:variability_mode] = PAYOFF
payoff_params[:β] = 0.9
payoff_params[:sigma] = 0.5

# # STRATEGY_MUTATION variable parameters
# strategy_params = copy(common_params)
# strategy_params[:variability_mode] = STRATEGY_MUTATION
# strategy_params[:initial_μ_s] = 0.05
# strategy_params[:β] = 0.8
# strategy_params[:sigma] = 0.045

# # RELATIONSHIP_MUTATION variable parameters
# relationship_params = copy(common_params)
# relationship_params[:variability_mode] = RELATIONSHIP_MUTATION
# strategy_params[:initial_μ_r] = 0.05
# relationship_params[:β] = 0.8
# relationship_params[:sigma] = 0.045

# Paramベクトルの生成
# param_dic_vec = [stable_params, population_params, payoff_params, strategy_params, relationship_params]
param_dic_vec = [stable_params, population_params, payoff_params]
param_vec_vec = [[Param(; param..., rng = MersenneTwister(seed)) for seed in 1:trial] for param in param_dic_vec]

# Modelベクトルの生成
model_vec_vec = [[Model(param) for param in param_vec] for param_vec in param_vec_vec];

In [4]:
println(Dates.format(now(), "Y-m-d H:M:S"))

# 989.715908 seconds (8.12 G allocations: 4.322 TiB, 51.75% gc time, 0.99% compilation time)
# 1169.336859 seconds (8.12 G allocations: 4.323 TiB, 58.49% gc time, 0.95% compilation time)
@time result_vec_vec = fetch.(Threads.@spawn [run(model) for model in model_vec] for model_vec in model_vec_vec);

println(Dates.format(now(), "Y-m-d H:M:S"))

2024-1-29 17:2:2
model.param.variability_mode = POPULATION
model.param.variability_mode = POPULATION
model.param.variability_mode = PAYOFF
model.param.variability_mode = POPULATION
model.param.variability_mode = PAYOFF
model.param.variability_mode = POPULATION
model.param.variability_mode = POPULATION
model.param.variability_mode = PAYOFF
model.param.variability_mode = POPULATION
model.param.variability_mode = POPULATION
model.param.variability_mode = PAYOFF
model.param.variability_mode = POPULATION
model.param.variability_mode = POPULATION
model.param.variability_mode = PAYOFF
model.param.variability_mode = POPULATION
model.param.variability_mode = POPULATION
model.param.variability_mode = POPULATION
model.param.variability_mode = PAYOFF
model.param.variability_mode = POPULATION
model.param.variability_mode = POPULATION
model.param.variability_mode = PAYOFF
model.param.variability_mode = POPULATION
model.param.variability_mode = POPULATION
model.param.variability_mode = PAYOFF
model.p

In [5]:
println(Dates.format(now(), "Y-m-d H:M:S"))

# jld2_file_snippets = ["stable", "population", "payoff", "strategy", "relationship"]
jld2_file_snippets = ["stable", "population", "payoff"]

# 641.446896 seconds (14.43 M allocations: 685.024 MiB, 0.14% gc time, 1.61% compilation time: <1% of which was recompilation)
@time Threads.@threads for (jld2_file_snippet, result_vec) in collect(zip(jld2_file_snippets, result_vec_vec))
    @save "result_$(jld2_file_snippet)_vec.jld2" compress=true result_vec
end

println(Dates.format(now(), "Y-m-d H:M:S"))

2024-1-29 18:14:59
168.223145 seconds (5.99 M allocations: 347.278 MiB, 0.47% gc time, 14.29% compilation time: <1% of which was recompilation)
2024-1-29 18:17:48
